# Final Project
# Recognition of fruits and vegetables and integrating a calorie counting function.
Made by: A.Nurman; K.Sherkhan



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# dataset : https://drive.google.com/drive/folders/1E5tbjL-gDqD2P2yBHY_C5w7GtSFir7O7?usp=drive_link
# Importing necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define constants
IMAGE_WIDTH, IMAGE_HEIGHT = 150, 150
BATCH_SIZE = 32
NUM_EPOCHS = 10

# Define directories for train and validation data
base_dir = '/content/drive/MyDrive/Final_images'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Define data augmentation and preprocessing for train and validation data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow validation images in batches using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(20, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=NUM_EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

# Save the model
model.save('fruit_vegetable_classification_model.h5')

# Define the calorie counts for each class
calorie_counts = {
    'Apple': 95,
    'Banana': 105,
    'Orange': 62
}

# Function to calculate calories based on predicted class
def calculate_calories(image_path):
    # Load the saved model
    loaded_model = tf.keras.models.load_model('fruit_vegetable_classification_model.h5')

    # Preprocess the image
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array /= 255.0

    # Predict the class
    predictions = loaded_model.predict(img_array)

    # Map class index to class label
    class_labels = ['Apple', 'Banana', 'Orange']
    predicted_class_index = np.argmax(predictions[0])
    predicted_class = class_labels[predicted_class_index]

    # Check if predicted class is in the calorie_counts dictionary
    if predicted_class in calorie_counts:
        calorie_count = calorie_counts[predicted_class]
    else:
        calorie_count = 0

    return calorie_count

# Example usage:
image_path = '/content/drive/MyDrive/Final_images/train/Apple/App_045.jpg'
calories = calculate_calories(image_path)
print("Calories:", calories)


Found 503 images belonging to 20 classes.
Found 477 images belonging to 20 classes.
Epoch 1/10
15/15 [==============================] - 234s 16s/step - loss: 3.1278 - accuracy: 0.0807 - val_loss: 2.9519 - val_accuracy: 0.1652
Epoch 2/10
15/15 [==============================] - 63s 4s/step - loss: 2.7028 - accuracy: 0.1699 - val_loss: 2.3886 - val_accuracy: 0.2478
Epoch 3/10
15/15 [==============================] - 63s 4s/step - loss: 2.1829 - accuracy: 0.2781 - val_loss: 1.7385 - val_accuracy: 0.3817
Epoch 4/10
15/15 [==============================] - 63s 4s/step - loss: 1.7511 - accuracy: 0.4161 - val_loss: 1.7358 - val_accuracy: 0.4353
Epoch 5/10
15/15 [==============================] - 53s 4s/step - loss: 1.2403 - accuracy: 0.5817 - val_loss: 1.1521 - val_accuracy: 0.6071
Epoch 6/10
15/15 [==============================] - 53s 4s/step - loss: 1.1738 - accuracy: 0.5499 - val_loss: 1.2218 - val_accuracy: 0.6384
Epoch 7/10
15/15 [==============================] - 53s 4s/step - loss: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 240ms/step
Calories: 95


In [ ]:
model.save("/content/model.h5")
!ls -lha

total 437M
drwxr-xr-x 1 root root 4.0K Mar  6 16:47 .
drwxr-xr-x 1 root root 4.0K Mar  6 15:56 ..
drwxr-xr-x 4 root root 4.0K Mar  4 14:27 .config
drwx------ 5 root root 4.0K Mar  6 16:15 drive
-rw-r--r-- 1 root root 218M Mar  6 16:46 fruit_vegetable_classification_model.h5
-rw-r--r-- 1 root root 218M Mar  6 16:47 model.h5
drwxr-xr-x 1 root root 4.0K Mar  4 14:28 sample_data


In [ ]:
!pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 45.1 MB/s eta 0:00:00


In [ ]:
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, '/content/model_data')

In [ ]:
from google.colab import files

files.download("/content/model_data/model.json")
files.download("/content/model_data/group1-shard1of19.bin")
files.download("/content/model_data/group1-shard2of19.bin")
files.download("/content/model_data/group1-shard3of19.bin")
files.download("/content/model_data/group1-shard4of19.bin")
files.download("/content/model_data/group1-shard5of19.bin")
files.download("/content/model_data/group1-shard6of19.bin")
files.download("/content/model_data/group1-shard7of19.bin")
files.download("/content/model_data/group1-shard8of19.bin")
files.download("/content/model_data/group1-shard9of19.bin")
files.download("/content/model_data/group1-shard10of19.bin")
files.download("/content/model_data/group1-shard11of19.bin")
files.download("/content/model_data/group1-shard12of19.bin")
files.download("/content/model_data/group1-shard13of19.bin")
files.download("/content/model_data/group1-shard14of19.bin")
files.download("/content/model_data/group1-shard15of19.bin")
files.download("/content/model_data/group1-shard16of19.bin")
files.download("/content/model_data/group1-shard17of19.bin")
files.download("/content/model_data/group1-shard18of19.bin")
files.download("/content/model_data/group1-shard19of19.bin")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>